import necessary libraries

In [1]:
from imutils import build_montages
import numpy as np
import cv2
import pickle
from matplotlib import pyplot as plt
import random

make pair in datasets like mnist,cifar10,imagenet,...

In [2]:
# percentage of each technique for make same pairs
# different pairs in mnist are same for all states
same_pairs = [80,10,10] #[same class, rotate, noise]
def convert_to_percentage():
  assert sum(same_pairs) == 100
  return (same_pairs[0],same_pairs[0]+same_pairs[1],same_pairs[0]+same_pairs[1]+same_pairs[2])

def advance_make_pairs(images, labels, count):
  pairImages = []
  pairLabels = []
  numClasses = len(np.unique(labels))
  idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
  counter_1 = int(count / (2))
  pos_counter = 0
  neg_counter = 0
  same_label_counter = 0
  rotate_counter = 0
  noise_counter = 0
  same_percentages = convert_to_percentage()
  for i in range(counter_1):
    choose_augment = random.randint(1,100)
    label = np.random.randint(0,numClasses)
    first_img_pos = np.random.choice(idx[label])
    first_img = images[first_img_pos]
    if choose_augment <= same_percentages[0]:
      second_img_pos = np.random.choice(idx[label])
      second_img = images[second_img_pos]
      pos_counter += 1
      same_label_counter+=1
    elif choose_augment <= same_percentages[1]:
      image_center = tuple(np.array(first_img.shape[1::-1]) / 2)
      angle = random.randint(-10,10)
      rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
      second_img = cv2.warpAffine(first_img, rot_mat, first_img.shape[1::-1], flags=cv2.INTER_LINEAR)
      pos_counter += 1
      rotate_counter+=1
    elif choose_augment <= same_percentages[2]:
      second_img =  cv2.GaussianBlur(first_img, (3, 3), 0)
      pos_counter += 1
      noise_counter+=1
      #print('noise')
    pairImages.append([first_img, second_img])
    pairLabels.append([1])
  counter_2 = count - counter_1
  i=0
  while i < counter_2:
    for j in range(10):
      for z in range(j+1,10):
        first_label = j
        second_label = z
        first_img_pos = np.random.choice(idx[first_label])
        second_img_pos = np.random.choice(idx[second_label])
        first_img = images[first_img_pos]
        second_img = images[second_img_pos]
        pairImages.append([first_img, second_img])
        pairLabels.append([0])
        neg_counter += 1
        i+=1
        if (i==counter_2):
          break
      if (i==counter_2):
        break
    if (i == counter_2):
      break
  print('**********')
  print(f'total positive pairs:{pos_counter} equal to {(pos_counter/count)*100}')
  print(f'total negagtive pairs:{neg_counter} equal to {(neg_counter/count)*100}')
  print(f'total same_label :{same_label_counter} equal to {(same_label_counter/pos_counter)*100} of positives')
  print(f'total rotate :{rotate_counter} equal to {(rotate_counter/pos_counter)*100} of positives')
  print(f'total noise :{noise_counter} equal to {(noise_counter/pos_counter)*100} of positives')
  print('********')
  return (np.array(pairImages), np.array(pairLabels))




make pair in our dataset(khorma)

In [ ]:
#            [same class,  180 rotate,    noise,     0-flie,      1-flip]
same_pairs = [15,60,5,10,10]
#                 [salem kharab,salem mkharab, msalem kharab, salem msalem, msalem mkharab, mkharab kharab]
different_pairs = [45,20,20,5,5,5]
def convert_to_percentage_same():
  assert sum(same_pairs) == 100
  return (same_pairs[0],same_pairs[0]+same_pairs[1],same_pairs[0]+same_pairs[1]+same_pairs[2],same_pairs[0]+same_pairs[1]+same_pairs[2]+same_pairs[3],
          same_pairs[0]+same_pairs[1]+same_pairs[2]+same_pairs[3]+same_pairs[4])
def convert_to_percentage_diff():
  assert sum(different_pairs) == 100
  result = []
  for i in range(len(different_pairs)):
    result.apppend(sum(l[:i]))
  return result
def khorma_advance_make_pairs(images, labels, count):
  pairImages = []
  pairLabels = []
  numClasses = len(np.unique(labels))
  idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
  counter_1 = int(count / (2))
  pos_counter = 0
  neg_counter = 0
  same_label_counter = 0
  rotate_counter = 0
  flip_counter = 0
  noise_counter = 0
  same_pair = convert_to_percentage_same()
  for i in range(counter_1):
    choose_augment = np.random.randint(1,100)
    label = np.random.randint(0,numClasses)
    first_img_pos = np.random.choice(idx[label])
    first_img = images[first_img_pos]
    if choose_augment <= same_pair[0]:
      second_img_pos = np.random.choice(idx[label])
      second_img = images[second_img_pos]
      pos_counter += 1
      same_label_counter+=1
    elif choose_augment <= same_pair[1]:
      second_img = cv2.rotate(first_img, cv2.ROTATE_180)
      pos_counter += 1
      rotate_counter+=1
      #print('rotate')
    elif choose_augment <= same_pair[2]:
      second_img =  cv2.GaussianBlur(first_img, (7, 7), 0)
      pos_counter += 1
      noise_counter+=1
      #print('noise')
    elif choose_augment <=same_pair[3]:
      second_img = cv2.flip(first_img, 0)
      pos_counter += 1
      flip_counter+=1
      #print('flip')
    elif choose_augment<=same_pair[4]:
      second_img = cv2.flip(first_img, 1)
      pos_counter += 1
      flip_counter += 1
    pairImages.append([first_img, second_img])
    pairLabels.append([1])
  counter_2 = count - counter_1
  i=0
  dif_pairs = convert_to_percentage_diff()
  while i < counter_2:
    choose_neg_pair = np.random.randint(1,100)
    if choose_neg_pair <= dif_pairs[0]:
      first_label = 0
      second_label =3
    elif choose_neg_pair <= dif_pairs[1]:
      first_label = 0
      second_label = 2
    elif choose_neg_pair <= dif_pairs[2]:
      first_label = 1
      second_label = 3
    elif choose_neg_pair <= dif_pairs[3]:
      first_label = 0
      second_label = 1
    elif choose_neg_pair <= dif_pairs[4]:
      first_label = 1
      second_label = 2
    elif choose_neg_pair <= dif_pairs[5]:
      first_label = 2
      second_label = 3
    first_img_pos = np.random.choice(idx[first_label])
    second_img_pos = np.random.choice(idx[second_label])
    first_img = images[first_img_pos]
    second_img = images[second_img_pos]
    pairImages.append([first_img, second_img])
    pairLabels.append([0])
    neg_counter += 1
    i+=1
    if (i==counter_2):
      break
  print('**********')
  print(f'total positive pairs:{pos_counter} equal to {(pos_counter/count)*100}')
  print(f'total negagtive pairs:{neg_counter} equal to {(neg_counter/count)*100}')
  print(f'total same_label :{same_label_counter} equal to {(same_label_counter/pos_counter)*100} of positives')
  print(f'total rotate :{rotate_counter} equal to {(rotate_counter/pos_counter)*100} of positives')
  print(f'total flip :{flip_counter} equal to {(flip_counter/pos_counter)*100} of positives')
  print(f'total noise :{noise_counter} equal to {(noise_counter/pos_counter)*100} of positives')
  print('********')
  return (np.array(pairImages), np.array(pairLabels))

load dataset

In [ ]:
import tensorflow
mnist = tensorflow.keras.datasets.mnist
(x_train,y_train), (x_valid, y_valid) = mnist.load_data()

configs

In [4]:
import os

IMG_SHAPE = (28, 28,1)
BATCH_SIZE = 20
EPOCHS = 10
BASE_OUTPUT = "output"
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "siamese_model"])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])

define your model here

In [5]:
# import neccessary packages
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Flatten, BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
import tensorflow

def build_siamese_model(inputShape, embeddingDim=70):
  inputs = Input(inputShape)
  x = Conv2D(64, (7, 7), activation='relu', padding='valid')(inputs)

  x = Conv2D(64, (5, 5), activation='relu', padding='valid')(x)

  x = Conv2D(128, (5, 5), activation='relu', padding='valid')(x)

  x = Conv2D(128, (5, 5), activation='relu', padding='valid')(x)

  x = Conv2D(128, (5, 5), activation='relu', padding='valid')(x)

  x = Conv2D(128, (3, 3), activation='relu', padding='valid')(x)

  x = Conv2D(256, (3, 3), activation='relu', padding='valid')(x)

  pooledOutput = GlobalAveragePooling2D()(x)
  outputs = Dense(embeddingDim)(pooledOutput)
  model = Model(inputs, outputs)
	# return the model to the calling function
  model.summary()
  return model


utils

In [6]:
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
def euclidean_distance(vectors):
	# unpack the vectors into separate lists
	(featsA, featsB) = vectors
	# compute the sum of squared distances between the vectors
	sumSquared = K.sum(K.square(featsA - featsB), axis=1,
		keepdims=True)
	# return the euclidean distance between the vectors
	return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def plot_training(H, plotPath):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(H.history["loss"], label="train_loss")
	plt.plot(H.history["val_loss"], label="val_loss")
	plt.plot(H.history["accuracy"], label="train_acc")
	plt.plot(H.history["val_accuracy"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)

create dataset

In [ ]:
# import packages
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
import numpy as np
import cv2
train_size = 5000 # define number of train data
test_size = 100 # define number of test data
x_train = x_train / 255.0
x_valid = x_valid / 255.0
(pairTrain, labelTrain) = advance_make_pairs(x_train, y_train, train_size)
(pairTest, labelTest) = advance_make_pairs(x_train,y_train, test_size)


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


fit model with dataset

In [ ]:
from keras.api._v2.keras import callbacks
import keras
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                                              patience=5, min_lr=1e-10)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
imgA = Input(shape=IMG_SHAPE)
imgB = Input(shape=IMG_SHAPE)
featureExtractor = build_siamese_model(IMG_SHAPE,70)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)
model.compile(loss="binary_crossentropy", optimizer="adam",
	metrics=["accuracy"])
model.summary()
history = model.fit(
	[pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
	validation_data=([pairTest[:, 0], pairTest[:, 1]], labelTest[:]),
	batch_size=BATCH_SIZE,
	epochs=50,
	callbacks = [reduce_lr,early_stopping])
model.save(MODEL_PATH)
plot_training(history, PLOT_PATH)
model.save_weights('./siamese_network.h5') # you can define your specific path here

In [ ]:
p=model.predict([pairTest[:, 0], pairTest[:, 1]])

store train and validation features for more executaion speed

In [ ]:
train_featured = model.layers[2].predict(x_train)
valid_featured = model.layers[2].predict(x_valid)
np.save('./train_features.npy', train_featured) # define your path here
np.save('./valid_features.npy', valid_featured) # define your path here